In [1]:
import os
from langchain_core.tools import StructuredTool

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.llms import openai


load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

model = ChatOpenAI(model="gpt-3.5-turbo")

ModuleNotFoundError: No module named 'langchain_community'

In [6]:
from web3 import Web3
from eth_account import Account

# Connect to Ethereum blockchain
WORLDCHAIN_URL = os.getenv("WORLDCHAIN_RPC_URL")
web3 = Web3(Web3.HTTPProvider(WORLDCHAIN_URL))

# Check connection
if not web3.is_connected():
    raise ConnectionError("Failed to connect to the blockchain.")

# Wallet setup
PRIVATE_KEY = os.getenv("WALLET_PRIVATE_KEY")

assert PRIVATE_KEY is not None, "You must set PRIVATE_KEY environment variable"
assert PRIVATE_KEY.startswith("0x"), "Private key must start with 0x hex prefix"

ACCOUNT =  Account.from_key(PRIVATE_KEY)


In [ ]:
from langchain.agents import Tool
from langchain.prompts import PromptTemplate
from langchain.agents import create_react_agent, AgentExecutor,AgentType, initialize_agent

# Define tools for the agent
def get_wallet_balance(*args):
    balance = web3.eth.get_balance(ACCOUNT.address)
    return f"Wallet Balance: {web3.from_wei(balance, 'ether')} ETH"

#print(ACCOUNT.address)
#balanche= (web3.eth.get_balance(ACCOUNT.address))
#print(web3.from_wei(balanche, 'ether'))


# Tools list
tools = [
    Tool(name="Check Balance", func=get_wallet_balance, description="Get the wallet's current balance."),
]
# Define an agent prompt

template= ''' """You are a crypto wallet assistant. if you can not answer the Question you are asked use the tools provided to you. Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''''


prompt = PromptTemplate.from_template(template)

wallet_agent= create_react_agent(model,tools,prompt)
agent_executor = AgentExecutor(agent=wallet_agent, tools=tools)

# Initialize the agent
agent = initialize_agent(
  tools,
  llm=model,
  agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
  verbose=False,
)


In [8]:
# agent.run("what is my wallet balance")
agent_executor.invoke({"input":"What is my wallet balance"})

{'input': 'What is my wallet balance',
 'output': 'Your wallet balance is 0.999999818964994442 ETH.'}